In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
csv_path= os.path.join("Resources/lc_2016_2017.csv")
lending_club_data = pd.read_csv(csv_path, low_memory=False)

lending_club_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759338 entries, 0 to 759337
Data columns (total 72 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           759338 non-null  int64  
 1   member_id                    0 non-null       float64
 2   loan_amnt                    759338 non-null  int64  
 3   funded_amnt                  759338 non-null  int64  
 4   funded_amnt_inv              759338 non-null  float64
 5   term                         759338 non-null  object 
 6   int_rate                     759338 non-null  float64
 7   installment                  759338 non-null  float64
 8   grade                        759338 non-null  object 
 9   sub_grade                    759338 non-null  object 
 10  emp_title                    708368 non-null  object 
 11  emp_length                   708975 non-null  object 
 12  home_ownership               759338 non-null  object 
 13 

In [3]:
# Cleaning uneccesary data (pt.1)

lending_club_data.drop(['id','member_id','desc','zip_code','policy_code','addr_state','issue_d','initial_list_status','application_type'],axis=1,inplace=True)

lending_club_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759338 entries, 0 to 759337
Data columns (total 63 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    759338 non-null  int64  
 1   funded_amnt                  759338 non-null  int64  
 2   funded_amnt_inv              759338 non-null  float64
 3   term                         759338 non-null  object 
 4   int_rate                     759338 non-null  float64
 5   installment                  759338 non-null  float64
 6   grade                        759338 non-null  object 
 7   sub_grade                    759338 non-null  object 
 8   emp_title                    708368 non-null  object 
 9   emp_length                   708975 non-null  object 
 10  home_ownership               759338 non-null  object 
 11  annual_inc                   759338 non-null  float64
 12  verification_status          759338 non-null  object 
 13 

In [4]:
# Cleaning uneccesary data (pt.2)

# Converting null values to 0, or None depending on the type of data

lending_club_data[['dti_joint','annual_inc_joint','il_util' , 'mths_since_rcnt_il', 'open_acc_6m', 'open_il_12m', 'open_il_24m', 'total_bal_il', 'inq_last_12m' , 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl' ]] = lending_club_data[['dti_joint','annual_inc_joint','il_util' , 'mths_since_rcnt_il', 'open_acc_6m', 'open_il_12m', 'open_il_24m', 'total_bal_il', 'inq_last_12m' , 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl']].fillna(0)


lending_club_data[['tot_coll_amt','tot_cur_bal','total_rev_hi_lim']] = lending_club_data[['total_rev_hi_lim', 'tot_cur_bal', 'tot_coll_amt']].fillna(0)

lending_club_data.verification_status_joint = lending_club_data.verification_status_joint.fillna('None')

# Converting NaN values to 0 or 1 for the following columns

lending_club_data.mths_since_last_record = lending_club_data.mths_since_last_record.apply(lambda x: 0 if pd.isna(x) else 1) 
lending_club_data.mths_since_last_major_derog = lending_club_data.mths_since_last_major_derog.apply(lambda x: 0 if pd.isna(x) else 1) 
lending_club_data.mths_since_last_delinq = lending_club_data.mths_since_last_delinq.apply(lambda x: 0 if pd.isna(x) else 1)

# Drop the following five columns

lending_club_data.drop(['next_pymnt_d' ,'last_pymnt_d','last_credit_pull_d', 'emp_title','title'],axis=1,inplace=True) 

lending_club_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759338 entries, 0 to 759337
Data columns (total 58 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    759338 non-null  int64  
 1   funded_amnt                  759338 non-null  int64  
 2   funded_amnt_inv              759338 non-null  float64
 3   term                         759338 non-null  object 
 4   int_rate                     759338 non-null  float64
 5   installment                  759338 non-null  float64
 6   grade                        759338 non-null  object 
 7   sub_grade                    759338 non-null  object 
 8   emp_length                   708975 non-null  object 
 9   home_ownership               759338 non-null  object 
 10  annual_inc                   759338 non-null  float64
 11  verification_status          759338 non-null  object 
 12  loan_status                  759338 non-null  object 
 13 

In [5]:
# Save emp_length (Employment Length) unique values into emp_length_unique DataFrame

emp_length_unique = lending_club_data[['emp_length']].value_counts().to_frame('total')

emp_length_unique


,total
emp_length,
10+ years,257944
2 years,69494
< 1 year,62724
3 years,60843
1 year,50585
5 years,46139
4 years,45573
6 years,32616
8 years,28707


In [6]:
lending_club_data.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,759338.000000,759338.000000,759338.000000,759338.000000,759338.000000,7.593380e+05,758983.000000,759338.000000,759337.000000,759338.000000,...,7.593380e+05,759338.000000,759338.000000,759338.000000,759338.000000,759338.000000,759338.000000,759338.000000,759338.000000,759338.000000
mean,14707.775260,14707.738385,14702.529098,13.187041,442.584639,7.996778e+04,18.980803,0.357795,0.538299,0.522341,...,3.571929e+04,60.741616,1.339469,2.847862,5789.830192,59.250400,255.295804,0.968484,1.475109,2.107288
std,9215.456493,9215.456467,9213.003636,5.054311,275.739578,1.634141e+05,13.363398,0.951763,0.838306,0.499501,...,4.324025e+04,32.390009,1.521405,2.622538,5580.483017,20.485302,2370.673461,1.484022,2.681318,2.402572
min,1000.000000,1000.000000,775.000000,5.320000,30.120000,0.000000e+00,-1.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7500.000000,7500.000000,7500.000000,9.750000,241.700000,4.800000e+04,12.260000,0.000000,0.000000,0.000000,...,9.309000e+03,44.000000,0.000000,1.000000,2368.000000,46.000000,0.000000,0.000000,0.000000,0.000000
50%,12000.000000,12000.000000,12000.000000,12.620000,370.840000,6.700000e+04,18.180000,0.000000,0.000000,1.000000,...,2.360800e+04,69.000000,1.000000,2.000000,4416.000000,60.000000,0.000000,0.000000,0.000000,1.000000
75%,20000.000000,20000.000000,20000.000000,15.590000,590.505000,9.500000e+04,24.780000,0.000000,1.000000,1.000000,...,4.634000e+04,84.000000,2.000000,4.000000,7500.000000,74.000000,0.000000,1.000000,2.000000,3.000000
max,40000.000000,40000.000000,40000.000000,30.990000,1719.830000,1.100000e+08,999.000000,42.000000,5.000000,1.000000,...,1.711009e+06,1000.000000,28.000000,60.000000,776843.000000,211.000000,932461.000000,48.000000,111.000000,49.000000
